# Working with time series data

In [ ]:
import pandas as pd
%pylab inline
try:
    import seaborn
except ImportError:
    pass
pd.options.display.max_rows = 8

## Case study: air quality data of European monitoring stations (AirBase)

AirBase (The European Air quality dataBase): hourly measurements of all air quality monitoring stations from Europe. 

I downloaded and preprocessed some of the data ([python-airbase](https://github.com/jorisvandenbossche/python-airbase)): `data/airbase_data.csv`. This file includes the hourly concentrations of NO2 for 4 different measurement stations:

- FR04037 (PARIS 13eme): urban background site at Square de Choisy
- FR04012 (Paris, Place Victor Basch): urban traffic site at Rue d'Alesia
- BETR802: urban traffic site in Antwerp, Belgium
- BETN029: rural background site in Houtem, Belgium

See http://www.eea.europa.eu/themes/air/interactive/no2

## Importing the data

In [ ]:
!head -5 data/airbase_data.csv

As you can see, the missing values are indicated by `-9999`. This can be recognized by `read_csv` by passing the `na_values` keyword:

In [ ]:
data = pd.read_csv('data/airbase_data.csv', index_col=0, parse_dates=True, na_values=[-9999])
# or https://github.com/AlessandroChecco/pandas-tutorial/blob/master/data/airbase_data.csv?raw=true

## Exploring the data - recap of some useful methods

Some useful methods:

`head` and `tail`

In [48]:
data.head(3)

,BETR801,BETN029,FR04037,FR04012,months
1999-01-01 00:00:00,27.0,13.0,68.0,105.0,1
1999-01-01 01:00:00,33.0,15.0,70.0,116.0,1
1999-01-01 02:00:00,24.0,16.0,72.0,114.0,1


In [ ]:
data.tail()

In [ ]:
data.info()

Getting some basic summary statistics about the data with `describe`:

In [ ]:
data.describe()

Quickly visualizing the data

In [ ]:
data.plot(kind='box', ylim=[0,250]);

In [ ]:
data['BETR801'].plot(kind='hist', bins=50);

In [ ]:
data.plot(figsize=(12,6));

This does not say too much ..

We can select part of the data (eg the latest 500 data points):

In [ ]:
data[-500:].plot(figsize=(12,6));

Or we can use some more advanced time series features -> next section!

## Working with time series data

When we ensure the DataFrame has a `DatetimeIndex`, time-series related functionality becomes available:

In [ ]:
data.index

Indexing a time series works with strings:

In [ ]:
data["2010-01-01 09:00":"2010-01-01 12:00"]

A nice feature is **"partial string" indexing**, where we can do implicit slicing by providing a partial datetime string.

E.g. all data of 2012:

In [ ]:
data['2012']

Normally you would expect this to access a column named '2012', but as for a DatetimeIndex, pandas also tries to interprete it as a datetime slice.

Or all data of January up to March 2012:

In [ ]:
data['2012-01':'2012-03']

Time and date components can be accessed from the index:

In [ ]:
data.index.hour

In [ ]:
data.index.year

<div class="alert alert-success">
    <b>EXERCISE</b>: select all data starting from 1999
</div>

In [ ]:
data = data['1999':]

<div class="alert alert-success">
    <b>EXERCISE</b>: select all data in January for all different years
</div>

In [ ]:
data[data.index.month == 1]

<div class="alert alert-success">
    <b>EXERCISE</b>: select all data in January, February and March for all different years
</div>

In [ ]:
data['months'] = data.index.month
data[data['months'].isin([1, 2, 3])]

<div class="alert alert-success">
    <b>EXERCISE</b>: select all 'daytime' data (between 8h and 20h) for all days
</div>

In [ ]:
data[(data.index.hour >= 8) & (data.index.hour < 20)]

Another solution:

In [ ]:
data.between_time('08:00', '20:00')

## The power of pandas: `resample`

A very powerfull method is **`resample`: converting the frequency of the time series** (e.g. from hourly to daily data).

The time series has a frequency of 1 hour. I want to change this to daily:

In [ ]:
data = data['1999':]
data.resample('D').mean().head()

Similar to `groupby`, other methods can also be specified:

In [ ]:
data.resample('D').max().head()

The string to specify the new time frequency: http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases  
These strings can also be combined with numbers, eg `'10D'`.

Further exploring the data:

In [ ]:
data.resample('M').mean().plot(); # 'A' , 'D'

<div class="alert alert-success">
    <b>QUESTION</b>: plot the monthly mean and median concentration of the 'FR04037' station for the years 2009-2012
</div>

In [ ]:
data.loc['2009':, 'FR04037'].resample('M').mean().plot();
data.loc['2009':, 'FR04037'].resample('M').median().plot();

In [ ]:
data.loc['2009':, 'FR04037'].resample('M').agg(['mean', 'median']).plot();

<div class="alert alert-success">
    <b>QUESTION</b>: plot the monthly mininum and maximum daily concentration of the 'BETR801' station
</div>

In [ ]:
daily = data['FR04037'].resample('D').mean()

In [ ]:
daily.resample('M').agg(['min', 'max']).plot();

<div class="alert alert-success">
    <b>QUESTION</b>: make a bar plot of the mean of the stations in year of 2012
</div>

In [ ]:
data['2012'].mean().plot(kind='bar');

<div class="alert alert-success">
    <b>QUESTION</b>: The evolution of the yearly averages with, and the overall mean of all stations (indicate the overall mean with a thicker black line)?
</div>

In [ ]:
data.resample('A').mean().plot();
data.mean(axis=1).resample('A').mean().plot(color='k', linestyle='--', linewidth=4);

## Combination with groupby

`resample` can actually be seen as a specific kind of `groupby`. E.g. taking annual means with `data.resample('A', 'mean')` is equivalent to `data.groupby(data.index.year).mean()` (only the result of `resample` still has a DatetimeIndex).



In [ ]:
data.groupby(data.index.year).mean().plot();

But, `groupby` is more flexible and can also do resamples that do not result in a new continuous time series, e.g. by grouping by the hour of the day to get the diurnal cycle.

<div class="alert alert-success">
    <b>QUESTION</b>: how does the *typical monthly profile* look like for the different stations?
</div>

1\. add a column to the dataframe that indicates the month (integer value of 1 to 12):

In [ ]:
data['month'] = data.index.month

2\. Now, we can calculate the mean of each month over the different years:

In [ ]:
data.groupby('month').mean()

3\. plot the typical monthly profile of the different stations:

In [ ]:
data.groupby('month').mean().plot();

<div class="alert alert-success">
    <b>QUESTION</b>: plot the weekly 95% percentiles of the concentration in 'BETR801' and 'BETN029' for 2011 (Tip: use dropna first)
</div>

In [ ]:
df2011 = data['2011'].dropna()
df2011.groupby(df2011.index.week)[['BETN029', 'BETR801']].quantile(0.95).plot();

In [ ]:
df2011[['BETN029', 'BETR801']].resample('W').agg(lambda x: x.quantile(0.95)).plot();

---
Let's delete the column we created first

In [ ]:
data = data.drop('month', axis=1)

<div class="alert alert-success">
    <b>QUESTION</b>: The typical diurnal profile for the different stations?
</div>

In [ ]:
data.groupby(data.index.hour).mean().plot();

<div class="alert alert-success">
    <b>QUESTION</b>: What are the number of exceedances of hourly values above the European limit 200 µg/m3 for each year/station?
</div>

In [ ]:
exceedances = data > 200

In [ ]:
# group by year and count exceedances (sum of boolean)
exceedances = exceedances.groupby(exceedances.index.year).sum()

In [ ]:
exceedances

In [ ]:
ax = exceedances.loc[2005:].plot(kind='bar');
ax.axhline(18, color='k', linestyle='--');

<div class="alert alert-success">
    <b>QUESTION</b>: And are there exceedances of the yearly limit value of 40 µg/m3 since 2000 ?
</div>

In [ ]:
yearly = data['2000':].resample('A')

In [ ]:
(yearly > 40).sum()

In [ ]:
yearly.plot();
plt.axhline(40, linestyle='--', color='k');

<div class="alert alert-success">
    <b>QUESTION</b>: The maximum daily 8 hour mean should be below 100 µg/m³. What are the number of exceedances of this limit for each year/station?
</div>

Tip: have a look at the `rolling` method to perform moving window operations.

In [ ]:
exceedances = data.rolling(8).mean().resample('D').max() > 100
exceedances = exceedances.groupby(exceedances.index.year).sum()
ax = exceedances.loc[2005:].plot(kind='bar');

<div class="alert alert-success">
    <b>QUESTION</b>: Calculate the correlation between the different stations
</div>


In [ ]:
data[['BETR801', 'BETN029', 'FR04037', 'FR04012']].corr()

In [ ]:
data[['BETR801', 'BETN029', 'FR04037', 'FR04012']].resample('D').corr()